In [ ]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()


0% [Working]
            
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease

0% [Connecting to archive.ubuntu.com] [Connecting to security.ubuntu.com (185.1
                                                                               
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease

0% [Connecting to archive.ubuntu.com (185.125.190.36)] [Waiting for headers] [W
                                                                               
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:4 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:7 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:8 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:9 http://ppa.lau

In [1]:
# Install pyspark
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=5afca2b2aa5e5783ff939e9ee32bef10c65aed61485dc3534199c0d7d6ed28eb
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [35]:
# Import packages
from pyspark.sql import SparkSession
import time
from pyspark.sql.functions import avg, year

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [59]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
home_sales_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", inferSchema=True, header=True)

home_sales_df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [60]:
# 2. Create a temporary view of the DataFrame.
home_sales_df.createOrReplaceTempView("home_sales")

In [79]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
query_1 = """
SELECT year(date) AS Year_Sold, bedrooms, round(mean(price),2) AS Average_Price
FROM home_sales
GROUP BY Year_Sold, bedrooms
HAVING bedrooms = 4
ORDER BY Year_Sold
"""
spark.sql(query_1).show()

+---------+--------+-------------+
|Year_Sold|bedrooms|Average_Price|
+---------+--------+-------------+
|     2019|       4|     300263.7|
|     2020|       4|    298353.78|
|     2021|       4|    301819.44|
|     2022|       4|    296363.88|
+---------+--------+-------------+



In [81]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
query_2 = """
SELECT date_built AS Year_built, bedrooms, bathrooms, round(mean(price),2) as Average_Price
FROM home_sales
GROUP BY Year_built, bedrooms, bathrooms
HAVING bedrooms = 3 AND bathrooms = 3
ORDER BY Year_built
"""
spark.sql(query_2).show()

+----------+--------+---------+-------------+
|Year_built|bedrooms|bathrooms|Average_Price|
+----------+--------+---------+-------------+
|      2010|       3|        3|    292859.62|
|      2011|       3|        3|    291117.47|
|      2012|       3|        3|    293683.19|
|      2013|       3|        3|    295962.27|
|      2014|       3|        3|    290852.27|
|      2015|       3|        3|     288770.3|
|      2016|       3|        3|    290555.07|
|      2017|       3|        3|    292676.79|
+----------+--------+---------+-------------+



In [88]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
query_3 = """
SELECT date_built AS Year_built, bedrooms, bathrooms, floors, max(sqft_living) AS sqft_living, round(mean(price),2) As Average_Price
FROM home_sales
GROUP BY Year_built, bedrooms, bathrooms, floors
HAVING bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
ORDER BY Year_built
"""
spark.sql(query_3).show()

+----------+--------+---------+------+-----------+-------------+
|Year_built|bedrooms|bathrooms|floors|sqft_living|Average_Price|
+----------+--------+---------+------+-----------+-------------+
|      2010|       3|        3|     2|       3945|    280447.23|
|      2011|       3|        3|     2|       4827|    281413.45|
|      2012|       3|        3|     2|       5412|    295858.68|
|      2013|       3|        3|     2|       5043|    295142.13|
|      2014|       3|        3|     2|       5226|    294195.13|
|      2015|       3|        3|     2|       5404|    291788.36|
|      2016|       3|        3|     2|       5275|     287812.6|
|      2017|       3|        3|     2|       5235|    282026.59|
+----------+--------+---------+------+-----------+-------------+



In [92]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.
start_time = time.time()

query_4 = """
SELECT view, round(mean(price),2) AS Average_Price
FROM home_sales
GROUP BY view
HAVING Average_Price >= 350000
ORDER BY view DESC
"""

spark.sql(query_4).show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|Average_Price|
+----+-------------+
| 100|    1026669.5|
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
+----+-------------+
only showing top 20 rows

--- 0.5836246013641357 seconds ---


In [93]:
# 7. Cache the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [94]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [96]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()

query_4_cached = """
SELECT view, round(mean(price),2) AS Average_Price
FROM home_sales
GROUP BY view
HAVING Average_Price >= 350000
ORDER BY view DESC
"""

spark.sql(query_4_cached).show()

print("--- %s seconds ---" % (time.time() - start_time))

# Comparison: The runtime for the cached query was faster than the runtime for the uncached query!

+----+-------------+
|view|Average_Price|
+----+-------------+
| 100|    1026669.5|
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
+----+-------------+
only showing top 20 rows

--- 0.4268648624420166 seconds ---


In [97]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
home_sales_df.write.partitionBy("date_built").mode("overwrite").parquet("home_sales_parquet")

In [98]:
# 11. Read the parquet formatted data.
parquet_partition_df =spark.read.parquet('home_sales_parquet')

In [99]:
# 12. Create a temporary table for the parquet data.
parquet_partition_df.createOrReplaceTempView('par_par_home_sales')

In [101]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()

query_4_parquet = """
SELECT view, round(mean(price),2) AS Average_Price
FROM par_par_home_sales
GROUP BY view
HAVING Average_Price >= 350000
ORDER BY view DESC
"""

spark.sql(query_4_parquet).show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|Average_Price|
+----+-------------+
| 100|    1026669.5|
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
+----+-------------+
only showing top 20 rows

--- 0.6976616382598877 seconds ---


In [102]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [103]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached("home_sales")

False